In [0]:
# Importando módulo de seguimiento
import sys
import importlib
sys.path.append('/Workspace/Users/luisfdiaz@bcp.com.pe/ScoreGuardian')

import core.SegScore as rmm
importlib.reload(rmm)

# Librerias y fuciones generales
from pyspark.sql.functions import date_format, expr, to_date, date_sub, add_months, col, when, coalesce, trim, broadcast, avg, max, min, lit, concat, window, round as colround, upper, abs as sparkabs
from pyspark.sql import functions as F
from pyspark import StorageLevel
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.sql.window import Window
from pyspark.sql.types import StructType, StructField, StringType

#Mostrar los duplicados
def print_res(sparkf_df):
  import pandas as pd
  pd.set_option('display.max_rows', None)
  pd.set_option('display.max_columns', None)
  pd_spark_df = sparkf_df.toPandas()
 
  return pd_spark_df

In [0]:
# Importando base y configuraciones iniciales
base_seguimiento = spark.sql("select distinct * from catalog_lhcl_prod_bcp.bcp_edv_fabseg.t39290_hm_app_hiptrad_f")
base_seguimiento = base_seguimiento.withColumn("CODMES", col("CODMES").cast("integer"))
base_seguimiento = base_seguimiento.withColumn("DEF5", when(col("DISTANCIA_DEF24") <= 5, lit(1)).otherwise(lit(0)))
base_seguimiento = base_seguimiento.withColumn("DEF4", when(col("DISTANCIA_DEF24") <= 4, lit(1)).otherwise(lit(0))) 

base_seguimiento.persist(StorageLevel.MEMORY_AND_DISK).count()
base_seguimiento = base_seguimiento.dropDuplicates(["CODCLAVEOPECTA"])

# Verificando duplicandos en el segundo Join
dup = base_seguimiento.groupBy("CODCLAVEOPECTA", "CODMES").count().alias("count").filter(col("count")>1)
print_res(dup)

In [0]:
pd_rbm='PD_RBM'
pd_ant='PD_APP_HIP_TRDPYME_2Q24_REP_SUITE' 
pd_cal='PD_APP_HIP_TRDPYME_2Q24_REP_FAB'
pd_trc='XB_ADC_POND85_F'
tipo_banda = 'Jeffrey'#Jeffrey Vasiseck
codmes_default = 202307
columna_monto='MTOAPROBADO_SOLES'
titulo='Calibración Applicant App Tradicional'
condiciones= "CODMES >= 201901 AND DEF_120_BLOQ_REF_0 = 0"

In [0]:
# Objeto de Seguimiento
Obj_Seg = rmm.MonitorScore(
  nombre_modelo         = 'Personas Modelo Applicant Tradicional',
  codigo_modelo         = 'MOD-BCP-20658',
  tier                  = 'II',
  detalle_seguimiento   = 'Seguimiento Applicant Tradicional',
  mes_seguimiento       = '202504',
  base_monitoreo        = base_seguimiento, #dataframe de spark de tu base de seguimiento 
  pd1                   = pd_cal, #
  pd2                   = pd_rbm, #
  # pd3                   = pd_rbm, #
  monto_credito         = columna_monto, #
  query_univ            = "DEF_120_BLOQ_REF_0=0 and codmes >202201", #
  bandas                = tipo_banda, 
  codmes_default        = codmes_default, #
  meses_ventana_target  = 24, #  ventana de target
  meses_atraso_target   = 4  #  condicion de target
)

# 1. Calibración

In [0]:
Obj_Seg_cal, rc_1, rm_1, fc_1, fm_1 = Obj_Seg.EvolCal(
  # Configuración de filtro de uso y pd de calibración
  query_uso          = "CODMES>=202106 and segmento_banca_f='1. Afluente'",
  pd_seg             = pd_cal,
  pd_comparacion     = True,
  proys_def_temp     = 6,
  proys_mor_temp     = 0,
  rd_aux             = 'RD21',
  rd_aux2            = 'RD12', 
  mora_temp          = 'MORA_30_3',

  # Configuración de proyecciones por cuentas
  proy_def_temp_xc   = "Best_proy",
  suav_def_temp_xc   = 1,
  proy_mor_temp_xc   = 'det1_xc',
  suav_mor_temp_xc   = 1,
  prof_hist_xc       = 14,

  # Configuración de proyecciones por montos
  proy_def_temp_xm   = "np.where(met2_xm==0, met2_xm+0.002, met2_xm)",
  suav_def_temp_xm   = 1,
  proy_mor_temp_xm   = 'det1_xm',
  suav_mor_temp_xm   = 1,
  prof_hist_xm       = 14,

  # Detalles del gráfico de calibración
  titulo             = titulo,
  pos_leyenda        = (0.5, -0.25),
  eje_y_xc           = 0.05,
  eje_y_xm           = 0.05,
  dim_grafico        = (32, 7),
  punt_mora          = 85, #Tamaño de puntos de mora temprana
  etiquetas          =True,
  pos_etiquetas_xm   = 10,
  pos_etiquetas_xc   = 10,
  tamaño_etiqueta    = 14,

  # Exportar factores para vista driver
  exportar_factores  = True
)

In [0]:
rc_1

In [0]:
rm_1

In [0]:
fm_1

In [0]:
Obj_Seg_cal, rc_2, rm_2, fc_2, fm_2 = Obj_Seg.EvolCal(
  # Configuración de filtro de uso y pd de calibración
  query_uso          = "CODMES>=202105 and segmento_banca_f<>'1. Afluente'",
  pd_seg             = pd_cal,
  pd_comparacion     = True,
  proys_def_temp     = 6,
  proys_mor_temp     = 0,
  rd_aux             = 'RD21',
  rd_aux2            = 'RD12', 
  mora_temp          = 'MORA_30_3',

  # Configuración de proyecciones por cuentas
  proy_def_temp_xc   = "Best_proy",
  suav_def_temp_xc   = 1,
  proy_mor_temp_xc   = 'det1_xc',
  suav_mor_temp_xc   = 1,
  prof_hist_xc       = 14,

  # Configuración de proyecciones por montos
  proy_def_temp_xm   = "np.where(met1_xm==0, met1_xm+0.005, np.where(calxmontos['CODMES']==202311, met1_xm*1, met1_xm))",
  suav_def_temp_xm   = 1,
  proy_mor_temp_xm   = 'det1_xm',
  suav_mor_temp_xm   = 1,
  prof_hist_xm       = 14,

  # Detalles del gráfico de calibración
  titulo             = titulo,
  pos_leyenda        = (0.5, -0.25),
  eje_y_xc           = 0.13,
  eje_y_xm           = 0.13,
  dim_grafico        = (32, 7),
  punt_mora          = 85, #Tamaño de puntos de mora temprana
  etiquetas          =True,
  pos_etiquetas_xm   = 10,
  pos_etiquetas_xc   = 10,
  tamaño_etiqueta    = 14,

  # Exportar factores para vista driver
  exportar_factores  = True
)

In [0]:
rm_2

In [0]:
rc_2

In [0]:
fm_2

In [0]:
Obj_Seg_cal, rc, rm, fc, fm = Obj_Seg.EvolCal(
  # Configuración de filtro de uso y pd de calibración
  query_uso          = "CODMES>=202101",
  pd_seg             = pd_cal,
  pd_comparacion     = True,
  proys_def_temp     = 6,
  proys_mor_temp     = 0,
  rd_aux             = 'RD21',
  rd_aux2            = 'RD12', # Corregido
  mora_temp          = 'MORA_30_3',

  # Configuración de proyecciones por cuentas
  proy_def_temp_xc   = 'det1_xm', 
  suav_def_temp_xc   = 1,
  proy_mor_temp_xc   = 'det1_xm', 
  suav_mor_temp_xc   = 1,
  prof_hist_xc       = 14,

  # Configuración de proyecciones por montos
  proy_def_temp_xm   = "det1_xm", 
  suav_def_temp_xm   = 1,
  proy_mor_temp_xm   = 'det1_xm', 
  suav_mor_temp_xm   = 1,
  prof_hist_xm       = 14,
  fact_to_proy_xc    = [fc_1, fc_2],
  fact_to_proy_xm    = [fm_1, fm_2],

  # Detalles del gráfico de calibración
  titulo             = titulo,
  pos_leyenda        = (0.5, -0.25),
  eje_y_xc           = 0.05,
  eje_y_xm           = 0.05,
  dim_grafico        = (32, 7),
  punt_mora          = 85, 
  etiquetas          =True,
  pos_etiquetas_xm   =10,
  pos_etiquetas_xc   =10,
  tamaño_etiqueta    =14,

  # Exportar factores para vista driver
  exportar_factores  = True
)

In [0]:
rc

In [0]:
rm

In [0]:
driver = 'segmento_banca_f'
Obj_SegPSI, mix_n, mix_m, psi_c, psi_m = Obj_Seg.MixPSI(
  # Selección del driver y filtro de uso
  driver             = driver,
  query_uso          = 'CODMES >= 202101',
  cast_int           = False,

  # Ventana de construcción o más antigua
  codmes_inicio1     = 202201,
  codmes_fin1        = 202312,

  # Periodo reciente
  codmes_inicio2     = 202401,
  codmes_fin2        = 202412,

  # Detalles del gráfico
  titulo             = driver,
  dim_grafico        = (25,6.5),
  pos_leyenda        = (0.5, -0.25),
  borde              = 0.08
)

# 2. Discriminación

In [0]:
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn('PD_APP_HIP_TRDPYME_2Q24', col('PD_APP_HIP_TRDPYME_2Q24').cast('float'))
var_pd = [pd_cal, 'PD_APP_HIP_TRDPYME_2Q24', pd_rbm]
caldri_veh_app = Obj_Seg.EvolDisVar(
  # Parametros de figura individual
  query_filtro = '',
  codmes_ini   = 202206,
  codmes_fin   = 202312,
  amplitud     = 6,
  disc_target  = [24, 21,18],
  umbral_sup   = 0.5,
  umbral_inf   = 0.4,
  moviles      = True,
  dim_grafico  = (22.5, 6),
  pos_leyenda  = (0.5, -0.35),
  etiquetas    = True,
  gini_abs     = False,
 
  # Parametros de figura final
  ncolumnas = 3,
  nfilas    = 1,
  variables = var_pd,
  dim_grafico_total = (30, 6),
  ncol_leyenda = 5,
  vspacio=-0.25,
  hspacio=4,
  tamaño_etiqueta = 14 
)

In [0]:
var_pd = ['PD_APP_HIP_TRDPYME_2Q24', pd_cal]
caldri_veh_app = Obj_Seg.EvolDisVar(
  # Parametros de figura individual
  query_filtro = '',
  codmes_ini   = 202206,
  codmes_fin   = 202312,
  amplitud     = 6,
  disc_target  = [24, 21,18],
  umbral_sup   = 0.5,
  umbral_inf   = 0.4,
  moviles      = True,
  dim_grafico  = (22.5, 6),
  pos_leyenda  = (0.5, -0.35),
  etiquetas    = True,
  gini_abs     = False,
 
  # Parametros de figura final
  ncolumnas = 2,
  nfilas    = 1,
  variables = var_pd,
  dim_grafico_total = (20, 6),
  ncol_leyenda = 5,
  vspacio=-0.25,
  hspacio=4,
  tamaño_etiqueta = 14 
)

# 3. Seguimiento de variables del modelo


**Variable**                      | **Descripción**                                                                 | **Peso**  | **Sentido** |
|----------------------------------|----------------------------------------------------------------------------------|-----------|-------------|
| XB_ADC_POND85_F                  | Score base                                                                      | 62.19%    | +           |
| EXP_PCT_EVOL_SHIP_U6M_RT_U24_A3 | Evolución del promedio de deuda no hipotecaria entre 6 y 24 meses               | 11.18%    | +           |
| RCC_PCT_UTL6_UTL24_RT_U24_A2     | Evolución de la utilización promedio de líneas entre 6 y 24 meses               | 9.29%     | +           |
| SLD_PRM_PAS_MIN_24_24_RT_U24_I   | Ratio entre el pasivo mínimo y el pasivo promedio en los últ. 24 meses         | 5.28%     | +           |
| FLAG_PYME                        | Flag de Fondeo BCP Pyme                                                         | 5.07%     | -           |


In [0]:
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn('DEU_NOHIP_6_24', col('DEU_NOHIP_6_24').cast('float'))
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn('UTIL_PROM_6_24', col('UTIL_PROM_6_24').cast('float'))
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn('PAS_MIN_PROM24', col('PAS_MIN_PROM24').cast('float'))

# Corrigiendo deuda no hipotecaria
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn("DEU_NOHIP_6_24", when(col("DEU_NOHIP_6_24") <0, 0).otherwise(col("DEU_NOHIP_6_24")))

var_cal = ["XB_ADC_POND85_REP_SALESFORCE", "DEU_NOHIP_6_24", "UTIL_PROM_6_24", "PAS_MIN_PROM24"]
var_cal_Q = Obj_Seg.CutVar(variables=var_cal, num_cortes=4, query_corte="CODMES BETWEEN 202101 AND 202410 AND DEF_120_BLOQ_REF_0 = 0")

# Corrigiendo significado de missing
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn("DEU_NOHIP_6_24_Q", when(col("DEU_NOHIP_6_24_Q") == "98. Missing", "5. Sin deuda NOHIP").otherwise(col("DEU_NOHIP_6_24_Q")))
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn("UTIL_PROM_6_24_Q", when(col("UTIL_PROM_6_24_Q") == "98. Missing", "5. Sin utilización").otherwise(col("UTIL_PROM_6_24_Q")))

In [0]:
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn('RCC_PCT_UTL6_UTL24_RT_U24', col('RCC_PCT_UTL6_UTL24_RT_U24').cast('float'))
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn('EXP_PCT_EVOL_SHIP_U6M_RT_U24', col('EXP_PCT_EVOL_SHIP_U6M_RT_U24').cast('float'))
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn('SLD_PRM_PAS_MIN_24_24_RT_U24', col('SLD_PRM_PAS_MIN_24_24_RT_U24').cast('float'))

# Corrigiendo deuda no hipotecaria
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn("RCC_PCT_UTL6_UTL24_RT_U24", when(col("RCC_PCT_UTL6_UTL24_RT_U24") <0, 0).otherwise(col("RCC_PCT_UTL6_UTL24_RT_U24")))
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn("RCC_PCT_UTL6_UTL24_RT_U24", when(col("RCC_PCT_UTL6_UTL24_RT_U24") <0, 0).otherwise(col("RCC_PCT_UTL6_UTL24_RT_U24")))

var_cal1 = ["XB_ADC_POND85_REP_SALESFORCE", "EXP_PCT_EVOL_SHIP_U6M_RT_U24_1", "RCC_PCT_UTL6_UTL24_RT_U24_1", "SLD_PRM_PAS_MIN_24_24_RT_U24_1"]
var_cal1_Q = Obj_Seg.CutVar(variables=var_cal1, num_cortes=4, query_corte="CODMES BETWEEN 202101 AND 202410 AND DEF_120_BLOQ_REF_0 = 0")

# Corrigiendo significado de missing
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn("RCC_PCT_UTL6_UTL24_RT_U24_1_Q", when(col("RCC_PCT_UTL6_UTL24_RT_U24_1_Q") == "98. Missing", "5. Sin deuda NOHIP").otherwise(col("RCC_PCT_UTL6_UTL24_RT_U24_1_Q")))
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn("EXP_PCT_EVOL_SHIP_U6M_RT_U2_1_Q", when(col("EXP_PCT_EVOL_SHIP_U6M_RT_U24_1_Q") == "98. Missing", "5. Sin utilización").otherwise(col("EXP_PCT_EVOL_SHIP_U6M_RT_U24_1_Q")))

In [0]:
caldri = Obj_Seg.CalVar(
  # Parametros de figura individual
  query_uso          = "CODMES BETWEEN 202207 AND 202307",
  pd_seg             = pd_cal,
  rd_aux             = 'RD12',
  pd_comparacion     = False,
  factores_cuentas   = fc,
  factores_montos    = fm,
  mat_importancia    = 0, 
  cast_int           = True,
  dim_grafico        = (25, 6),
  pos_leyenda        = (0.5, -0.3),
  punt_mora          = 70,
  proys_def_temp     =6,
 
  # Parametros de figura final
  tipo_vista         = 'M',
  ncolumnas          = 2,
  nfilas             = 2,
  variables          = var_cal_Q,
  dim_grafico_total  = (16.5, 9),
  ncol_leyenda       = 5,
  etiquetas_total    = True, 
  pos_etiqueta       = 0.05,
  vspacio            = -0.5,
  hspacio            = 0.5,
  filtro_var         ="not like '%issing%'",
  iv                 = True 
)

In [0]:
caldri = Obj_Seg.CalVar(
  # Parametros de figura individual
  query_uso          = "CODMES BETWEEN 202207 AND 202307",
  pd_seg             = pd_cal,
  rd_aux             = 'RD12',
  pd_comparacion     = False,
  factores_cuentas   = fc,
  factores_montos    = fm,
  mat_importancia    = 0, 
  cast_int           = True,
  dim_grafico        = (25, 6),
  pos_leyenda        = (0.5, -0.3),
  punt_mora          = 70,
  proys_def_temp     =6,
 
  # Parametros de figura final
  tipo_vista         = 'M',
  ncolumnas          = 2,
  nfilas             = 2,
  variables          = var_cal1_Q,
  dim_grafico_total  = (16.5, 9),
  ncol_leyenda       = 5,
  etiquetas_total    = True, 
  pos_etiqueta       = 0.05,
  vspacio            = -0.5,
  hspacio            = 0.5,
  filtro_var         ="not like '%issing%'",
  iv                 = True 
)

In [0]:
psi_dri = Obj_Seg.MixPSIVar(
  # Parametros de figura individual
    query_uso         ='CODMES >= 202011',
    cast_int          =True,
    codmes_inicio1    =202112,
    codmes_fin1       =202204,
    codmes_inicio2    =202501,
    codmes_fin2       =202505,
    dim_grafico       =(25, 6),
    pos_leyenda       =(0.5, -0.35),
 
    # Parametros de figura final
    tipo_vista        = 'M',
    ncolumnas         = 2,
    nfilas            = 2,
    variables         = var_cal_Q,
    dim_grafico_total = (17, 10),
    ncol_leyenda      =5,
    tamaño_leyenda    = 8.25,
    vspacio           =-0.45,
    hspacio           =0,
    borde             = 0.08,
    metrica_secundaria='Media'
)

In [0]:
psi_dri = Obj_Seg.MixPSIVar(
  # Parametros de figura individual
    query_uso         ='CODMES >= 202011',
    cast_int          =True,
    codmes_inicio1    =202112,
    codmes_fin1       =202204,
    codmes_inicio2    =202501,
    codmes_fin2       =202505,
    dim_grafico       =(25, 6),
    pos_leyenda       =(0.5, -0.35),
 
    # Parametros de figura final
    tipo_vista        = 'M',
    ncolumnas         = 2,
    nfilas            = 2,
    variables         = var_cal1_Q,
    dim_grafico_total = (17, 10),
    ncol_leyenda      =5,
    tamaño_leyenda    = 8.25,
    vspacio           =-0.45,
    hspacio           =0,
    borde             = 0.08,
    # metrica_secundaria='Media'
)

In [0]:
gini = Obj_Seg.EvolDisVar(
  # Parametros de figura individual
    query_filtro = '',
    codmes_ini   = 202206,
    codmes_fin   = 202312,
    amplitud     = 6,
    disc_target  = [24, 21,18],
    umbral_sup   = 0.15,
    umbral_inf   = 0.05,
    #umbral_opc   = 0.1,
    moviles      = True,
    dim_grafico  = (15, 6),
    pos_leyenda  = (0.5, -0.35),
 
    etiquetas    = True,
    tamaño_etiqueta    =12,

    # Parametros de figura final
    ncolumnas = 2,
    nfilas    = 2,
    variables = var_cal,
    dim_grafico_total = (17, 10),
    ncol_leyenda      = 6,
    vspacio           =-0.25,
    hspacio           =5
  )

# 4. Drivers de negocio

In [0]:
from pyspark.sql.functions import date_format, concat, lit, lpad

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    'GRADO_ESTUDIO',
    when(col("MAX_GRADO")=='BACHILLER', '1. BACHILLER')
    .when(col("MAX_GRADO")=='TITULO', '3. TITULO')
    .when(col("MAX_GRADO")=='MAESTRIA', '2. MAESTRIA')
    .otherwise('99. Missing')
)

PD_SEG='PD_APP_HIP_TRDPYME_2Q24_REP_FAB'
PAUTA=335

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "SCR_XB",
    F.lit(174.24575241) + F.lit(57.707801636) * F.log((1 / col("PD_APP_HIP_TRDPYME_2Q24_REP_FAB")) - 1)
)


sol_credit = spark.table("catalog_lhcl_prod_bcp.bcp_udv_int_vu.m_solicitudcreditohipotecario")
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.join(
    sol_credit.select(
        concat(lit('2'), lpad(sol_credit.codsolicitud, 10, '0')).alias('codsolicitud'),
        col("destipfinalidadcreditohipotecario")
    ),
    on="codsolicitud",
    how='left'
)

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn("PAUTA", when(col('SCR_XB') >= PAUTA , 1)
                                            .when(col('SCR_XB') < PAUTA , 0)
                                            .when(col('destipfinalidadcreditohipotecario')=='MIGRACION DE OTRO BANCO', "97. CDD")
                                            .when(col('SCR_XB').isNull(), "99. Missing"))

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn("RANGO_PD_REP", when(col(PD_SEG).isNull(), '99. Missing')
                   .when(col(PD_SEG) <= 0.0045641, '01. [0 - 0.45%]')
                   .when(col(PD_SEG) <= 0.0151879, '02. <0.45 - 1.51%]')
                   .when(col(PD_SEG) <= 0.0353823, '03. <1.51 - 3.53%]')
                   .when(col(PD_SEG) <= 0.0581041, '04. <3.53 - 5.81%]')
                   .when(col(PD_SEG) > 0.0581041, '05. <5.81 - 100%]')
                   .when(col('destipfinalidadcreditohipotecario')=='MIGRACION DE OTRO BANCO', "97. CDD")
                   .otherwise('98. Otro'))

In [0]:
dri_neg = ["LTV_CORREGIDO_VF", "SUITE_AntLaboralMeses"]
dri_neg_Q = Obj_Seg.CutVar(variables=dri_neg, num_cortes=4, query_corte="CODMES BETWEEN 202101 AND 202410 AND DEF_120_BLOQ_REF_0 = 0")
dri_neg_Q = dri_neg_Q + ["MARCA_HML_T", "FLG_SOLTERO_ESPJ", "PLAZO_R", "SUITE_Edad_J_Q", "FLG_PDH_2_3", "PAUTA", "RANGO_PD_REP"]

In [0]:
caldri = Obj_Seg.CalVar(
  # Parametros de figura individual
  query_uso          = "CODMES BETWEEN 202212 AND 202306",
  pd_seg             = pd_cal,
  rd_aux             = 'RD12',
  pd_comparacion     = True,
  factores_cuentas   = fc,
  factores_montos    = fm,
  mat_importancia    = 0, 
  cast_int           = True,
  dim_grafico        = (25, 6),
  pos_leyenda        = (0.5, -0.3),
  punt_mora          = 70,
  proys_def_temp=6,
 
  # Parametros de figura final
  tipo_vista         = 'M',
  ncolumnas          = 3,
  nfilas             = 3,
  variables          = dri_neg_Q,
  dim_grafico_total  = (25, 15),
  ncol_leyenda       = 5,
  etiquetas_total    = True, 
  pos_etiqueta       = 0.05,
  vspacio            = -0.5,
  hspacio            = 0.5,
  filtro_var         ="not like '%issing%'",
  iv                 = True 
)

In [0]:
psi_dri = Obj_Seg.MixPSIVar(
  # Parametros de figura individual
    query_uso       ='CODMES >= 202201 AND CODMES <> 202407',
    cast_int        =True,
    codmes_inicio1  =202201,
    codmes_fin1     =202212,
    codmes_inicio2  =202501,
    codmes_fin2     =202505,
    dim_grafico     =(22.5, 6),
    pos_leyenda     =(0.5, -0.35),
 
    # Parametros de figura final
    tipo_vista      = 'M',
    ncolumnas       = 3,
    nfilas          = 3,
    variables       = dri_neg_Q,
    dim_grafico_total = (25, 15),
    ncol_leyenda    =8,
    tamaño_leyenda  = 8.25,
    vspacio         =-0.25,
    hspacio         =0,
    borde           = 0.08
)

In [0]:
dri_neg = ["PRS_TASA", "INGRESO_RBM"]
dri_neg_Q = Obj_Seg.CutVar(variables=dri_neg, num_cortes=4, query_corte="CODMES BETWEEN 202101 AND 202410 AND DEF_120_BLOQ_REF_0 = 0")

# POWER BI

In [0]:
import pandas as pd

def procesar_factores(n_SegProy, Mes_Def):
    df_factores = pd.DataFrame()  # Inicializar un DataFrame vacío
    
    for i in range(1, n_SegProy + 1):
        # Generar los nombres de los DataFrames dinámicamente
        fc = globals()[f'fc_{i}']
        fm = globals()[f'fm_{i}']
        
        # Añadir la columna SEG_PROYECCION
        fc['SEG_PROYECCION'] = i
        fc['CODMES_DEF'] = Mes_Def

        # Realizar la unión de los DataFrames
        df_resultante = pd.merge(fc, fm, on="CODMES", how="inner", suffixes=('_fc', '_fm'))

        # Seleccionar las columnas deseadas
        df_resultante = df_resultante[['CODMES_DEF','CODMES', 'SEG_PROYECCION', 'RD_PROY_DEF_fm', 'FAC_CAL_DRI_M', 'RD_PROY_DEF_fc', 'FAC_CAL_DRI_N']]

        # Reemplazar los valores de factores NaN por 0 y RD por 0.005
        df_resultante[['FAC_CAL_DRI_M', 'FAC_CAL_DRI_N']] = df_resultante[['FAC_CAL_DRI_M', 'FAC_CAL_DRI_N']].fillna(0)
        df_resultante[['RD_PROY_DEF_fm', 'RD_PROY_DEF_fc']] = df_resultante[['RD_PROY_DEF_fm', 'RD_PROY_DEF_fc']].fillna(0.005)

        # Cambiar los nombres de las columnas
        df_resultante.rename(columns={'RD_PROY_DEF_fm': 'proyeccion_montos'}, inplace=True)
        df_resultante.rename(columns={'FAC_CAL_DRI_M': 'factor_montos'}, inplace=True)
        df_resultante.rename(columns={'RD_PROY_DEF_fc': 'proyeccion_cuentas'}, inplace=True)
        df_resultante.rename(columns={'FAC_CAL_DRI_N': 'factor_cuentas'}, inplace=True)

        # Concatenar el DataFrame resultante al DataFrame final
        df_factores = pd.concat([df_factores, df_resultante], ignore_index=True)

    return df_factores

In [0]:
df_factores=procesar_factores(2,202505)
df_factores

In [0]:
df_factores = spark.createDataFrame(df_factores)
df_factores.createOrReplaceTempView("temp_view")
spark.sql("CREATE OR REPLACE TABLE catalog_lhcl_prod_bcp.bcp_edv_fabseg.T45988_FAC_ADM_HIP_TRAD_202506 LOCATION 'abfss://bcp-edv-fabseg@adlscu1lhclbackp05.dfs.core.windows.net/HIPOTECARIO_2025/APP/202506/T45988_FAC_ADM_HIP_TRAD_202503' AS SELECT * FROM temp_view")

In [0]:
# Definir la fecha base
MES_DEF_ACT = 202505
default = 24

# Convertir la fecha base a un objeto datetime
# base_date = datetime.strptime(MES_DEF_ACT, "%Y%m")

# Función para calcular los meses anteriores
def calculate_previous_months(codmes, cant):
  numeromes = (codmes // 100) * 12 + (codmes % 100)
  return ((numeromes - 1 + cant) // 12) * 100 + ((numeromes + 11 - 11 * cant) % 12) + 1

# Calcular los meses anteriores

codmes_a1 = calculate_previous_months(MES_DEF_ACT, -default+11)
codmes_a2 = calculate_previous_months(MES_DEF_ACT, -default+10)
codmes_a3 = calculate_previous_months(MES_DEF_ACT, -default+9)
codmes_a4 = calculate_previous_months(MES_DEF_ACT, -default+8)
codmes_a5 = calculate_previous_months(MES_DEF_ACT, -default+7)
codmes_a6 = calculate_previous_months(MES_DEF_ACT, -default+6)
codmes_a7 = calculate_previous_months(MES_DEF_ACT, -default+5)
codmes_a8 = calculate_previous_months(MES_DEF_ACT, -default+4)
codmes_a9 = calculate_previous_months(MES_DEF_ACT, -default+3)
codmes_a10 = calculate_previous_months(MES_DEF_ACT, -default+2)
codmes_a11 = calculate_previous_months(MES_DEF_ACT, -default+1)
codmes_a12 = calculate_previous_months(MES_DEF_ACT, -default)

monto = "MTOAPROBADO_SOLES"

In [0]:
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn("SEG_PROYECCION", when(Obj_Seg.base_monitoreo.SEGMENTO_BANCA_F == "1. Afluente", 1).when(Obj_Seg.base_monitoreo.SEGMENTO_BANCA_F != "1. Afluente", 2))

In [0]:
#CÁLCULO DE VALORES DE MESES Y CRUCE

from datetime import datetime, timedelta
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when

# Definir la fecha base
MES_DEF_ACT = 202505
default = 24

# Convertir la fecha base a un objeto datetime
# base_date = datetime.strptime(MES_DEF_ACT, "%Y%m")

# Función para calcular los meses anteriores
def calculate_previous_months(codmes, cant):
  numeromes = (codmes // 100) * 12 + (codmes % 100)
  return ((numeromes - 1 + cant) // 12) * 100 + ((numeromes + 11 - 11 * cant) % 12) + 1

# Calcular los meses anteriores

codmes_a1 = calculate_previous_months(MES_DEF_ACT, -default+11)
codmes_a2 = calculate_previous_months(MES_DEF_ACT, -default+10)
codmes_a3 = calculate_previous_months(MES_DEF_ACT, -default+9)
codmes_a4 = calculate_previous_months(MES_DEF_ACT, -default+8)
codmes_a5 = calculate_previous_months(MES_DEF_ACT, -default+7)
codmes_a6 = calculate_previous_months(MES_DEF_ACT, -default+6)
codmes_a7 = calculate_previous_months(MES_DEF_ACT, -default+5)
codmes_a8 = calculate_previous_months(MES_DEF_ACT, -default+4)
codmes_a9 = calculate_previous_months(MES_DEF_ACT, -default+3)
codmes_a10 = calculate_previous_months(MES_DEF_ACT, -default+2)
codmes_a11 = calculate_previous_months(MES_DEF_ACT, -default+1)
codmes_a12 = calculate_previous_months(MES_DEF_ACT, -default)

# Cargar las tablas para el cruce
df_a = Obj_Seg.base_monitoreo
df_b = spark.table("catalog_lhcl_prod_bcp.bcp_edv_fabseg.T45988_FAC_ADM_HIP_TRAD_202506")

# Crear la columna ult_rd
df_a = df_a.withColumn("ult_rd", when(col("CODMES") == codmes_a11, col("DEF23"))
                                 .when(col("CODMES") == codmes_a10, col("DEF22"))
                                 .when(col("CODMES") == codmes_a9, col("DEF22"))
                                 .when(col("CODMES") == codmes_a8, col("DEF21"))
                                 .when(col("CODMES") == codmes_a7, col("DEF20"))
                                 .when(col("CODMES") == codmes_a6, col("DEF19"))
                                 .when(col("CODMES") == codmes_a5, col("DEF18"))
                                 .when(col("CODMES") == codmes_a4, col("DEF17"))
                                 .when(col("CODMES") == codmes_a3, col("DEF16"))
                                 .otherwise(0))


# Realizar la unión izquierda
df_joined = df_a.join(df_b, (df_a.CODMES == df_b.CODMES) & (df_a.SEG_PROYECCION == df_b.SEG_PROYECCION), "left")

# Calcular las nuevas columnas
df_cal01 = df_joined.select(
    df_a["*"],
    df_b.factor_montos,
    df_b.factor_cuentas,
    (col("ult_rd") * df_b.factor_montos).alias("def12_proy_factor_m"),
    (col("ult_rd") * df_b.factor_cuentas).alias("def12_proy_factor_n"),
    df_b.proyeccion_montos.alias("def12_proy_valor_m"),
    df_b.proyeccion_cuentas.alias("def12_proy_valor_n")
)

# la última tabla creada aquí es df_cal01

#-------------------------------------------------------------------------------------------------------------------

#CUBO PRINCIPAL

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, when, count

# Crear las columnas flgventana_diagnostico, flgventana_reciente, flgdef_cerrado, flgdef_temp, flgmora_temp
df_cal01 = df_cal01.withColumn("flgventana_diagnostico", when(col("CODMES").between(codmes_a11, codmes_a6), 1).otherwise(0)) \
           .withColumn("flgventana_reciente", when(col("CODMES").between(202410, 202412), 1).otherwise(0)) \
           .withColumn("flgdef_cerrado", when(col("CODMES") <= codmes_a12, 1).otherwise(0)) \
           .withColumn("flgdef_temp", when(col("CODMES") <= codmes_a9, 1).otherwise(0)) \
           .withColumn("flgmora_temp", when(col("CODMES") <= 202401, 1).otherwise(0)) \
           .withColumn("PASA_PAUTA", when(col("SC_APP_TRAD") >= 335, 1).otherwise(0))
           
#Creación de nuevas PDs
# Calcular XB_TRAD a partir de CSOL_SC_RBM (SC_TRAD)
df_cal01 = df_cal01.withColumn('XB_TRAD', expr("(174.25 - CSOL_SC_RBM) / 57.708"))

# Calcular PD_TRAD a partir de XB_TRAD y renombrar la columna resultante a PD_RBM
df_cal01 = df_cal01.withColumn('PD_RBM', expr("1 / (1 + exp(-XB_TRAD))"))

# Calcular PD_TRAD para las columnas XB especificadas y renombrar las columnas resultantes
xb_columns = {
    'XB_ADC_POND85_F': 'PD_TRC',
    'XB_HIP_REP': 'PD_HIP',
    'XB_DEM_S_ADCW_REP': 'PD_DEMOGRAFICO',
    'XB_ACT_S_ADCW': 'PD_ACTIVO',
    'XB_TRX_S_ADCW': 'PD_TRANSACCIONAL',
    'XB_PAS_S_ADCW': 'PD_PASIVO',
    'XB_RCC_S_ADCW': 'PD_RCC'
}

for col_name, new_col_name in xb_columns.items():
    df_cal01 = df_cal01.withColumn(new_col_name, expr(f"1 / (1 + exp(-{col_name}))"))

# Eliminar la columna temporal XB_TRAD
df_cal01 = df_cal01.drop('XB_TRAD')

# LISTA DE NOMBRES DE COLUMNAS
lista_pd = [
    "PD_P7_AJUS_REP",
    "PD_APP_HIP_TRDPYME_2Q24_REP",
    "PD_TRC",
    "PD_RBM",
    "PD_HIP",
    "PD_DEMOGRAFICO",
    "PD_ACTIVO",
    "PD_TRANSACCIONAL",
    "PD_PASIVO",
    "PD_RCC"
]
# Crear una lista de expresiones de agregación
pd_result_xm = [sum(col(c) * col(monto)).alias(f"pd_{str(i+1).zfill(2)}_xm") for i, c in enumerate(lista_pd)]
# Crear una lista de expresiones de agregación sin multiplicar por monto
pd_result_xn = [sum(col(c)).alias(f"pd_{str(i+1).zfill(2)}_xn") for i, c in enumerate(lista_pd)]

#DEFINIR LISTA DE EJES 
lista_ejes = {
    "CAMPANIA_AGRUPADA": "EJE_P1",
    "TIPO_FONDEO": "EJE_P2",
    "SEGMENTO_BANCA_F": "EJE_P3",  
    "FLG_BANCARIZADO": "EJE_P4",
    "RANGO_SCORE": "EJE_P5",
    "DEU_NOHIP_6_24_Q": "EJE_S1",
    "UTIL_PROM_6_24_Q": "EJE_S2",
    "PAS_MIN_PROM24_Q": "EJE_S3",
    "ZONA_PAUTA5_F": "EJE_S4",
    "FLG_NODEP_ESPJ": "EJE_S5",
    "FLG_SOLTERO_ESPJ": "EJE_S6",
    "MONTO_R": "EJE_S7",
    "RAN_PLAZO": "EJE_S8",
    "PRS_TASA_Q": "EJE_S9",
    "NUMHIJOS1_Q": "EJE_S10",
    "EDAD_R": "EJE_S11",
    "RAN_ING_CY": "EJE_S12",
    "LTV_CORREGIDO_VF_Q": "EJE_S13",
    "INGRESO_RBM_Q": "EJE_S14",
    "MARCA_HML_T": "EJE_S15",
    "PASA_PAUTA": "EJE_S16"
}

# Crear una lista de expresiones de selección usando el diccionario
eje_result = [col(k).alias(v) for k, v in lista_ejes.items()]

# AGRUPAR Y AGREGAR DATOS
cubo_principal = df_cal01.groupBy(
    "CODMES",
    "seg_proyeccion",
    "flgventana_diagnostico",
    "flgventana_reciente",
    "flgdef_cerrado",
    "flgdef_temp",
    "flgmora_temp",
    col("SEGMENTO_BANCA_F").alias("EJE_TOOLTIP"),
    *eje_result,
).agg(
    *pd_result_xm,
    sum(col(monto)).alias("m"),
    sum(when(col("CODMES") > codmes_a12, 0).otherwise(col("DEF24") * col(monto))).alias("def_cerrado_xm"),
    sum(when(col("CODMES") > codmes_a9, 0).otherwise(col("DEF21") * col(monto))).alias("def_temp_xm"),
    sum(when(col("CODMES") > 202401, 0).otherwise(col("DEF12") * col(monto))).alias("mora_temp_xm"),
    sum(when(col("CODMES").between(codmes_a11, codmes_a6), col("def12_proy_factor_m") * col(monto)).otherwise(0)).alias("def_proy_factor_dt_xm"),
    sum(when(col("CODMES").between(codmes_a5, codmes_a3), col("def12_proy_factor_m") * col(monto)).otherwise(0)).alias("def_proy_factor_mt_xm"),
    sum(when(col("CODMES").between(codmes_a11, codmes_a6), col("def12_proy_valor_m") * col(monto)).otherwise(0)).alias("def_proy_valor_dt_xm"),
    sum(when(col("CODMES").between(codmes_a5, codmes_a3), col("def12_proy_valor_m") * col(monto)).otherwise(0)).alias("def_proy_valor_mt_xm"),
    *pd_result_xn,
    count(lit(1)).alias("n"),
    sum(when(col("CODMES") > codmes_a12, 0).otherwise(col("DEF24"))).alias("def_cerrado_xn"),
    sum(when(col("CODMES") > codmes_a9, 0).otherwise(col("DEF21"))).alias("def_temp_xn"),
    sum(when(col("CODMES") > 202401, 0).otherwise(col("DEF12"))).alias("mora_temp_xn"),
    sum(when(col("CODMES").between(codmes_a11, codmes_a6), col("def12_proy_factor_n")).otherwise(0)).alias("def_proy_factor_dt_xn"),
    sum(when(col("CODMES").between(codmes_a5, codmes_a3), col("def12_proy_factor_n")).otherwise(0)).alias("def_proy_factor_mt_xn"),
    sum(when(col("CODMES").between(codmes_a11, codmes_a6), col("def12_proy_valor_n")).otherwise(0)).alias("def_proy_valor_dt_xn"),
    sum(when(col("CODMES").between(codmes_a5, codmes_a3), col("def12_proy_valor_n")).otherwise(0)).alias("def_proy_valor_mt_xn")
)

cubo_principal_final = cubo_principal.toPandas()

#-------------------------------------------------------------------------------------------------------------------

#ETIQUETA DE EJES

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType


# Definir el esquema
schema = StructType([
    StructField("EJE", StringType(), True),
    StructField("ETIQUETA", StringType(), True)
])

# Crear los datos
data = [
    ("EJE_P1", "Producto"),
    ("EJE_P2", "Tipo de Fondeo"),
    ("EJE_P3", "Segmento Banca"),
    ("EJE_P4", "Flg Bancarizado"),
    ("EJE_P5", "Rango de Score"),
    ("EJE_S1", "Deuda promedio No Hipotecaria entre 6 y 24 m"),
    ("EJE_S2", "Utilizacion promedio de lineas entre 6 y 24 m"),
    ("EJE_S3", "Ratio entre el pasivo minimo y prom. en u24 m"),
    ("EJE_S4", "Zona Pauta"),
    ("EJE_S5", "Flag No Dependiente"),
    ("EJE_S6", "Flag Soltero"),
    ("EJE_S7", "Monto aprobado"),
    ("EJE_S8", "Rango Plazo"),
    ("EJE_S9", "Rango de Tasa"),
    ("EJE_S10", "Número de hijos"),
    ("EJE_S11", "Rango de Edad"),
    ("EJE_S12", "Rango Ingreso Conyugue"),
    ("EJE_S13", "LTV"),
    ("EJE_S14", "Rango de Ingreso Titular"),
    ("EJE_S15", "Marca HML"),
    ("EJE_S16", "Pasa Pauta")
]

# Crear el DataFrame
df_etiquetas_ejes_final = spark.createDataFrame(data, schema)

df_etiquetas_ejes = df_etiquetas_ejes_final.toPandas()

#-------------------------------------------------------------------------------------------------------------------

#ETIQUETA DE PDs

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType

# Crear una sesión de Spark
spark = SparkSession.builder.appName("SQLtoPySpark").getOrCreate()

# Definir el esquema
schema = StructType([
    StructField("PD", StringType(), True),
    StructField("DESCRIPTIVO", StringType(), True)
])

# Crear los datos
data = [
    ("01", "01-PD Vigente"),
    ("02", "02-PD Calibrado"),
    ("03", "03-PD Troncal"),
    ("04", "04-PD RBM"),
    ("05", "05-PD Hip"),
    ("06", "06-PD Demográfico"),
    ("07", "07-PD Activo"),
    ("08", "08-PD Transaccional"),
    ("09", "09-PD Pasivo"),
    ("10", "10-PD RCC"),
]

# Crear el DataFrame
df_descriptivo_pds_final = spark.createDataFrame(data, schema)
# Convertir a Pandas
df_descriptivo_pds = df_descriptivo_pds_final.toPandas()

In [0]:
# PASAR DE DATAFRAME A UNA TABLA DE DATABRICKS
cubo_principal.createOrReplaceTempView("temp_view")

spark.sql("""
CREATE OR REPLACE TABLE catalog_lhcl_prod_bcp.bcp_edv_fabseg.cubo_trad_202506
LOCATION 'abfss://bcp-edv-fabseg@adlscu1lhclbackp05.dfs.core.windows.net/HIPOTECARIO_2025/APP/202506/cubo_trad' 
AS SELECT * FROM temp_view
""")

In [0]:
#EXPORTAR EN ARCHIVO CSV (PARA EL CUBO_PRINCIPAL SE ESTÁ EXPORTANDO EN ZIP (COMPRIMIDO))

import zipfile

# Guardar los DataFrames como archivos CSV
df_etiquetas_ejes.to_csv("/Workspace/Users/luisfdiaz@bcp.com.pe/Seguimiento/Applicant Hipotecario/etiquetas_ejes.csv", index=False)
df_descriptivo_pds.to_csv("/Workspace/Users/luisfdiaz@bcp.com.pe/Seguimiento/Applicant Hipotecario/descriptivo_pds.csv", index=False)

# Guardar y comprimir cubo_principal_final
csv_path = "/Workspace/Users/luisfdiaz@bcp.com.pe/Seguimiento/Applicant Hipotecario/cubo_principal.csv"
zip_path = "/Workspace/Users/luisfdiaz@bcp.com.pe/Seguimiento/Applicant Hipotecario/cubo_principal.zip"

cubo_principal_final.to_csv(csv_path, index=False)

with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    zipf.write(csv_path, arcname="cubo_principal.csv")

In [0]:
cubo_principal_final